Q1(a): Deep Learning Models on MNIST/FashionMNIST
Step-by-Step Implementation Approach:
Setup and Imports: Import necessary libraries

Data Preparation: Load datasets with 70-10-20 split

Model Definition: Create ResNet-18 and ResNet-50 models

Training Pipeline: Implement training with validation

Hyperparameter Testing: Test all combinations

Results Collection: Store accuracy for comparison

In [18]:
# Run this cell first in Google Colab or Kaggle
!pip install torch torchvision thop
!pip install scikit-learn pandas numpy matplotlib
!pip install GPUtil psutil

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
from torch.cuda.amp import autocast, GradScaler
import time
from sklearn.metrics import accuracy_score
import pandas as pd
from torchvision.models import resnet18, resnet50

In [20]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [21]:
class ResNetMNIST(nn.Module):
    """Wrapper for ResNet to handle 1-channel MNIST images"""
    def __init__(self, model_type='resnet18', num_classes=10):
        super(ResNetMNIST, self).__init__()
        
        if model_type == 'resnet18':
            self.model = resnet18(pretrained=False)
            # Modify first conv layer for 1-channel input
            self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        elif model_type == 'resnet50':
            self.model = resnet50(pretrained=False)
            self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Modify final layer for 10 classes
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)


In [22]:
def load_datasets(dataset_name='MNIST'):
    """Load MNIST or FashionMNIST datasets"""
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # ResNet expects at least 32x32
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    if dataset_name == 'MNIST':
        dataset = torchvision.datasets.MNIST(
            root='./data', train=True, download=True, transform=transform
        )
        test_dataset = torchvision.datasets.MNIST(
            root='./data', train=False, download=True, transform=transform
        )
    else:  # FashionMNIST
        dataset = torchvision.datasets.FashionMNIST(
            root='./data', train=True, download=True, transform=transform
        )
        test_dataset = torchvision.datasets.FashionMNIST(
            root='./data', train=False, download=True, transform=transform
        )
    
    # Split train into train and validation (70%-10%-20%)
    train_size = int(0.7 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    
    train_dataset, val_dataset, _ = random_split(
        dataset, [train_size, val_size, test_size]
    )
    
    return train_dataset, val_dataset, test_dataset

In [23]:
def train_epoch(model, train_loader, optimizer, criterion, device, scaler, use_amp=True):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        if use_amp:
            with autocast():
                output = model(data)
                loss = criterion(output, target)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item()
        _, preds = torch.max(output, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(target.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    return running_loss / len(train_loader), accuracy

In [24]:

def validate(model, val_loader, criterion, device, use_amp=True):
    """Validate the model"""
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            
            if use_amp:
                with autocast():
                    output = model(data)
                    loss = criterion(output, target)
            else:
                output = model(data)
                loss = criterion(output, target)
            
            val_loss += loss.item()
            _, preds = torch.max(output, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(target.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    return val_loss / len(val_loader), accuracy

In [25]:
def test(model, test_loader, device, use_amp=True):
    """Test the model"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            if use_amp:
                with autocast():
                    output = model(data)
            else:
                output = model(data)
            
            _, preds = torch.max(output, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(target.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy


In [26]:
def run_experiment(dataset_name, model_type, batch_size, optimizer_name, lr, 
                   pin_memory=False, epochs=10, use_amp=True):
    """Run a complete experiment"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Load datasets
    train_dataset, val_dataset, test_dataset = load_datasets(dataset_name)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                             pin_memory=pin_memory, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                           pin_memory=pin_memory, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                            pin_memory=pin_memory, num_workers=2)
    
    # Create model
    model = ResNetMNIST(model_type=model_type, num_classes=10).to(device)
    
    # Define loss function
    criterion = nn.CrossEntropyLoss()
    
    # Define optimizer
    if optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    else:  # Adam
        optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Mixed precision training
    scaler = GradScaler() if use_amp else None
    
    # Training loop
    best_val_acc = 0
    train_times = []
    
    print(f"\nTraining {model_type} on {dataset_name}")
    print(f"Batch size: {batch_size}, Optimizer: {optimizer_name}, LR: {lr}")
    
    for epoch in range(epochs):
        start_time = time.time()
        
        # Train
        train_loss, train_acc = train_epoch(
            model, train_loader, optimizer, criterion, device, 
            scaler, use_amp
        )
        
        # Validate
        val_loss, val_acc = validate(model, val_loader, criterion, device, use_amp)
        
        epoch_time = time.time() - start_time
        train_times.append(epoch_time)
        
        print(f"Epoch {epoch+1}/{epochs}: "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
              f"Time: {epoch_time:.2f}s")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
    
    # Test
    test_acc = test(model, test_loader, device, use_amp)
    avg_train_time = np.mean(train_times)
    
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Average training time per epoch: {avg_train_time:.2f}s")
    
    return test_acc * 100, avg_train_time


In [27]:
# Run experiments for all hyperparameter combinations
def run_all_experiments(dataset_name='MNIST'):
    results = []
    
    # Hyperparameter combinations
    batch_sizes = [16, 32]
    optimizers = ['SGD', 'Adam']
    learning_rates = [0.001, 0.0001]
    models = ['resnet18', 'resnet50']
    epochs_list = [3, 5]  # Two different epoch counts
    
    for batch_size in batch_sizes:
        for optimizer_name in optimizers:
            for lr in learning_rates:
                for model_type in models:
                    for epochs in epochs_list:
                        # Test with pin_memory=True (better for GPU)
                        pin_memory = torch.cuda.is_available()
                        
                        print(f"\n{'='*60}")
                        print(f"Experiment: {model_type}, BS={batch_size}, "
                              f"Opt={optimizer_name}, LR={lr}, Epochs={epochs}")
                        print('='*60)
                        
                        try:
                            test_acc, train_time = run_experiment(
                                dataset_name=dataset_name,
                                model_type=model_type,
                                batch_size=batch_size,
                                optimizer_name=optimizer_name,
                                lr=lr,
                                pin_memory=pin_memory, 
                                epochs=epochs,
                                use_amp=True  # Constant as per requirements
                            )
                            
                            results.append({
                                'Dataset': dataset_name,
                                'Model': model_type,
                                'Batch_Size': batch_size,
                                'Optimizer': optimizer_name,
                                'Learning_Rate': lr,
                                'Epochs': epochs,
                                'Test_Accuracy_%': test_acc,
                                'Avg_Train_Time_s': train_time,
                                'Pin_Memory': pin_memory
                            })
                            
                        except Exception as e:
                            print(f"Error in experiment: {e}")
                            continue
    
    # Create results DataFrame
    df_results = pd.DataFrame(results)
    return df_results

In [ ]:
# Run experiments for MNIST
print("Running experiments for MNIST dataset...")
mnist_results = run_all_experiments('MNIST')

# Run experiments for FashionMNIST
print("\n\nRunning experiments for FashionMNIST dataset...")
fashion_results = run_all_experiments('FashionMNIST')

# Combine results
all_results = pd.concat([mnist_results, fashion_results], ignore_index=True)

# Save results
all_results.to_csv('experiment_results.csv', index=False)

# Display results in table format
print("\n\nResults Summary:")
print("="*100)

Running experiments for MNIST dataset...

Experiment: resnet18, BS=16, Opt=SGD, LR=0.001, Epochs=3
Using device: cuda

Training resnet18 on MNIST
Batch size: 16, Optimizer: SGD, LR: 0.001


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_55/167624454.py:31: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if use_amp else None
/tmp/ipykernel_55/56737039.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_55/2399673817.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please 

Epoch 1/3: Train Loss: 0.1805, Train Acc: 0.9437, Val Loss: 0.0571, Val Acc: 0.9833, Time: 37.09s


/tmp/ipykernel_55/56737039.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_55/2399673817.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 2/3: Train Loss: 0.0633, Train Acc: 0.9799, Val Loss: 0.0449, Val Acc: 0.9868, Time: 37.35s


...

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



Training resnet50 on FashionMNIST
Batch size: 16, Optimizer: SGD, LR: 0.0001


/tmp/ipykernel_55/167624454.py:31: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if use_amp else None
/tmp/ipykernel_55/56737039.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_55/2399673817.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/3: Train Loss: 1.0910, Train Acc: 0.6015, Val Loss: 0.6210, Val Acc: 0.7712, Time: 77.58s


/tmp/ipykernel_55/56737039.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


In [9]:
print(mnist_results)

[{'Dataset': 'MNIST', 'Model': 'resnet18', 'Batch_Size': 16, 'Optimizer': 'SGD', 'Learning_Rate': 0.001, 'Epochs': 3, 'Test_Accuracy_%': 98.81, 'Avg_Train_Time_s': 37.2}, {'Dataset': 'MNIST', 'Model': 'resnet18', 'Batch_Size': 16, 'Optimizer': 'SGD', 'Learning_Rate': 0.001, 'Epochs': 5, 'Test_Accuracy_%': 98.82, 'Avg_Train_Time_s': 37.06}, {'Dataset': 'MNIST', 'Model': 'resnet50', 'Batch_Size': 16, 'Optimizer': 'SGD', 'Learning_Rate': 0.001, 'Epochs': 3, 'Test_Accuracy_%': 98.48, 'Avg_Train_Time_s': 78.14}, {'Dataset': 'MNIST', 'Model': 'resnet50', 'Batch_Size': 16, 'Optimizer': 'SGD', 'Learning_Rate': 0.001, 'Epochs': 5, 'Test_Accuracy_%': 98.7, 'Avg_Train_Time_s': 77.51}, {'Dataset': 'MNIST', 'Model': 'resnet18', 'Batch_Size': 16, 'Optimizer': 'SGD', 'Learning_Rate': 0.0001, 'Epochs': 3, 'Test_Accuracy_%': 98.56, 'Avg_Train_Time_s': 36.39}, {'Dataset': 'MNIST', 'Model': 'resnet18', 'Batch_Size': 16, 'Optimizer': 'SGD', 'Learning_Rate': 0.0001, 'Epochs': 5, 'Test_Accuracy_%': 98.56, '

In [10]:
import pandas as pd

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(mnist_results)

# Display as a table
print(df)

   Dataset     Model  Batch_Size Optimizer  Learning_Rate  Epochs  \
0    MNIST  resnet18          16       SGD         0.0010       3   
1    MNIST  resnet18          16       SGD         0.0010       5   
2    MNIST  resnet50          16       SGD         0.0010       3   
3    MNIST  resnet50          16       SGD         0.0010       5   
4    MNIST  resnet18          16       SGD         0.0001       3   
5    MNIST  resnet18          16       SGD         0.0001       5   
6    MNIST  resnet50          16       SGD         0.0001       3   
7    MNIST  resnet50          16       SGD         0.0001       5   
8    MNIST  resnet18          16      Adam         0.0010       3   
9    MNIST  resnet18          16      Adam         0.0010       5   
10   MNIST  resnet50          16      Adam         0.0010       3   
11   MNIST  resnet50          16      Adam         0.0010       5   
12   MNIST  resnet18          16      Adam         0.0001       3   
13   MNIST  resnet18          16  

In [11]:
# Save to CSV
df.to_csv('mnist_results.csv', index=False)
print("Saved to 'mnist_results.csv'")

Saved to 'mnist_results.csv'
